In [ ]:
from langchain_ollama import ChatOllama

from langchain_core.prompts import (
                                        SystemMessagePromptTemplate,
                                        HumanMessagePromptTemplate,
                                        ChatPromptTemplate,
                                        MessagesPlaceholder
                                        )

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory

base_url = "http://localhost:11434"
model = 'llama3.2:3b'

llm = ChatOllama(base_url=base_url, model=model)

system = SystemMessagePromptTemplate.from_template("You are helpful assistant.")
human = HumanMessagePromptTemplate.from_template("{input}")


about = "My name is Laxmi Kant. I work for KGP Talkie."

messages = [system, MessagesPlaceholder(variable_name='history'), human]

prompt = ChatPromptTemplate(messages=messages)

chain = prompt | llm | StrOutputParser()

def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///chat_history.db")

runnable_with_history = RunnableWithMessageHistory(chain, get_session_history, 
                                                   input_messages_key='input', 
                                                   history_messages_key='history')

#chain.invoke({'prompt': about})

In [6]:
runnable_with_history = RunnableWithMessageHistory(chain, get_session_history)

In [5]:
def chat_with_llm(session_id, input):
    output = runnable_with_history.invoke(
        {'input': input},
        config={'configurable': {'session_id': session_id}}
    )

    return output

In [4]:
user_id = "laxmi_kant"
chat_with_llm(user_id, about)

c:\Users\Digi-Team\anaconda3\envs\ml\Lib\site-packages\langchain_core\runnables\history.py:606: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  message_history = self.get_session_history(


"It seems like we started again. You mentioned your name and that you work at KGP Talkies, but didn't elaborate further. Can you please tell me a bit more about what you do there? Are you involved in film production, distribution, or something else? I'm here to listen!"

In [26]:
user_id = "Herr_Mann"
chat_with_llm(user_id, "what is my name?")

"I think we've been here before! You told me your name is Herr Mann earlier. I'm just trying to help and remember the conversation. Is there anything else I can help you with?"

In [25]:
chat_with_llm(user_id, "My name is Herr Mann")

"A strong and handsome name! Unfortunately, I don't have any information about the person named Herr Mann, as this is our first conversation. Would you like to tell me a bit more about yourself, or would you like to chat about something else?"

In [ ]:
user_id = 'laxmi_kant'
history = get_session_history(user_id)

history.get_messages()

In [ ]:
history.clear()